In [2]:
!pip install --upgrade typing-extensions
!pip install --upgrade fastapi
!pip install boto3
!pip3 install pyngrok
!pip3 install uvicorn

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.

In [3]:
import pandas as pd
import numpy as np
from fastapi import FastAPI
from pathlib import Path
import torch
import boto3
from io import BytesIO

In [4]:


# Replace these values with your AWS credentials and bucket information
aws_access_key_id = 'AWS_ACCESS_KEY_ID'
aws_secret_access_key = 'AWS_SECRET_ACCESS_KEY'

# Set the S3 bucket and folder path
s3_bucket_name = 'team3-ass3'


# Create a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# Create an S3 client using the session
s3_client = session.client('s3')

# Specify the S3 object keys for download
image_ids_s3_key = f"image_ids.csv"
features_s3_key = f"features.npy"

# Download the files from S3
image_ids_obj = s3_client.get_object(Bucket=s3_bucket_name, Key=image_ids_s3_key)
features_obj = s3_client.get_object(Bucket=s3_bucket_name, Key=features_s3_key)

# Load the image IDs
image_ids = pd.read_csv(BytesIO(image_ids_obj['Body'].read()))['image_id'].tolist()
local_csv_path = 'image_ids.csv'
image_ids_df = pd.DataFrame({'image_id': image_ids})
image_ids_df.to_csv(local_csv_path, index=False)

# Load the features vectors
features = np.load(BytesIO(features_obj['Body'].read()))
np.save('features.npy', features)

# Convert features to Tensors: Float32 on CPU and Float16 on GPU
device = "cpu"  # Change this to "cuda" if using GPU
if device == "cpu":
    image_features = torch.from_numpy(features).float().to(device)
else:
    image_features = torch.from_numpy(features).to(device)

# Print some statistics
print(f"Images loaded: {len(image_ids)}")


Images loaded: 316


In [5]:
!pip install --upgrade git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-tu0wd1ts
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-tu0wd1ts
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369500 sha256=b2d52ed5dbac99ca4915491284f4f036df0ac4bba70a1557efe8853625ce5bf3
  Stored in directory: /tmp/pip-ephem-wheel-cache-gaziaf1v/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [27]:
from fastapi import FastAPI, HTTPException, File, UploadFile
from fastapi.responses import StreamingResponse
import io
import boto3
from PIL import Image
import torch
from torchvision import transforms
import clip
import base64
from scipy.spatial.distance import cosine

# Load the CLIP model
model, preprocess = clip.load("ViT-B/32", device=device)

s3_folder_path = 'image'
app = FastAPI()

def encode_search_query(search_query):
    print(f"Type of search_query: {type(search_query)}")
    with torch.no_grad():
        # Encode and normalize the search query using CLIP
        text_encoded = model.encode_text(clip.tokenize(search_query).to(device))
        text_encoded /= text_encoded.norm(dim=-1, keepdim=True)

    # Retrieve the feature vector
    #print(text_encoded)
    print(f"Text Encoded Type: {type(text_encoded)}")
    return text_encoded


def encode_image(image_bytes):
    # Encode and normalize the image using CLIP'
    image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    #image = Image.open(image_file).convert('RGB')
    image_input = preprocess(image).unsqueeze(0).to(device)
    image_features = model.encode_image(image_input)
    image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features


def find_best_matches(text_features, image_features, image_ids, results_count=3):
  # Compute the similarity between the search query and each image using the Cosine similarity
    similarities = (image_features @ text_features.T).squeeze(1)

    # Sort the images by their similarity score
    best_image_idx = (-similarities).argsort()

    # Return the image IDs of the best matches
    return [image_ids[i] for i in best_image_idx[:results_count]]

def find_similar_images(uploaded_image_features, image_features, image_ids, results_count=3):
    cos = torch.nn.CosineSimilarity(dim=0)

    # Calculate cosine similarities between the uploaded image and all other images
    similarities = cos(uploaded_image_features.unsqueeze(0), image_features)

    # Create a list of tuples containing similarity values and corresponding image indices
    similarities_with_indices = list(zip(similarities.tolist(), range(len(similarities))))

    # Sort the list based on similarity values in descending order
    sorted_similarities = sorted(similarities_with_indices, key=lambda x: x[0], reverse=True)

    # Take the top N indices
    best_image_indices = [i for _, i in sorted_similarities[:results_count]]
    print(sorted_similarities)
    print(type(sorted_similarities))

    # Return the image IDs of the best matches
    return [image_ids[i] for i in best_image_indices]




def search(search_query, image_features, image_ids, s3_bucket_name, s3_folder_path, results_count=3):
    # Encode the search query
    text_features = encode_search_query(search_query)

    # Find the best matches
    return find_best_matches(text_features, image_features, image_ids, results_count)


def search_by_image(image_file, image_features, image_ids,s3_bucket_name, s3_folder_path, results_count):
    # Encode the uploaded image
    image_bytes = base64.b64decode(image_file)

    uploaded_image_features = encode_image(image_bytes)
    # Find similar images

    return find_similar_images(uploaded_image_features, image_features, image_ids, results_count)

@app.get("/")
def read_root():
    return {"message": "Welcome to the image search API!"}

@app.post("/search_images_by_text/")
async def search_images_by_text(request:dict):
    search_query = request['search_query']
    results_count = request['results_count']

    result_image_ids = search(search_query, image_features, image_ids, s3_bucket_name, s3_folder_path, results_count)
    image_paths = [f"{s3_folder_path}/{image_id}.jpg" for image_id in result_image_ids]

    images = []
    for image_path in image_paths:
        image_bytes = download_image_from_s3(s3_bucket_name, image_path)
        images.append(image_bytes)

    def image_generator():
        for image_bytes in images:
            yield image_bytes
    return StreamingResponse(image_generator(), media_type="image/jpeg")

# @app.post("/search_images_by_image/")
# async def search_images_by_image(request:dict):

#       image_file = request.get('image_file')
#       results_count = request.get('results_count')
#       result_images = search_by_image(image_file, image_features, image_ids, s3_bucket_name, s3_folder_path,results_count)
#       image_paths = [f"{s3_folder_path}/{image_id}.jpg" for image_id in result_images]

#       images = []
#       for image_path in image_paths:
#           image_bytes = download_image_from_s3(s3_bucket_name, image_path)
#           images.append(image_bytes)
#           print(image_path)
#       print(len(images))
#       def image_generator():
#           for image_bytes in images:
#               yield image_bytes

#       return StreamingResponse(image_generator(), media_type="image/jpeg")

@app.post("/search_images_by_image/")
async def search_images_by_image(request:dict):

      image_file = request.get('image_file')
      results_count = request.get('results_count')
      result_images = search_by_image(image_file, image_features, image_ids, s3_bucket_name, s3_folder_path,results_count)
      image_paths = [f"{s3_folder_path}/{image_id}.jpg" for image_id in result_images]

      images = []
      for image_path in image_paths:
          image_bytes = download_image_from_s3(s3_bucket_name, image_path)
          images.append(image_bytes)
          print(image_path)
      print(len(images))

      # Use a custom separator to join image data
      custom_separator = b'YOUR_CUSTOM_SEPARATOR'
      joined_image_data = custom_separator.join(images)
      def image_generator():
          for image_bytes in images:
              yield joined_image_data

      return StreamingResponse(image_generator(), media_type="image/jpeg")

def download_image_from_s3(bucket_name, image_key):
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    image_bytes = s3.get_object(Bucket=bucket_name, Key=image_key)['Body'].read()
    return image_bytes

In [29]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

!ngrok authtoken ngrok-token

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Public URL: https://e161-35-196-84-171.ngrok-free.app


INFO:     Started server process [204]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     34.147.60.76:0 - "POST /search_images_by_image HTTP/1.1" 307 Temporary Redirect


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



image/den601_29_2_side.jpg
image/p101_3_back.jpg
image/15_2_side.jpg
3
INFO:     34.147.60.76:0 - "POST /search_images_by_image/ HTTP/1.1" 200 OK
INFO:     34.147.60.76:0 - "POST /search_images_by_image HTTP/1.1" 307 Temporary Redirect


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



image/den601_29_2_side.jpg
image/p101_3_back.jpg
image/15_2_side.jpg
3
INFO:     34.147.60.76:0 - "POST /search_images_by_image/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [204]
